In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

plt.style.use('ggplot')

import nltk

In [ ]:
# for google colab to use GPU for processing
# !pip install transformers
import torch
import tensorflow as tf
from transformers import pipeline
tf.test.gpu_device_name()

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

'/device:GPU:0'

In [ ]:
# mounting drive on colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# defining path for importing the csv file
path = f'/content/drive/MyDrive/Colab Notebooks/flip_clean.csv'
df = pd.read_csv(path)

In [ ]:
df.head()

,rating,review,comment,text
0,5,Perfect product!,Good product 👍👍👍👍👍 thanks FlipkartREAD MORE,Good product 👍👍👍👍👍 thanks Flipkart
1,5,Fabulous!,Nice product...But if I use it in three finger...,Nice product...But if I use it in three finger...
2,5,Awesome,Very nice product go fot it ❤️READ MORE,Very nice product go fot it ❤️
3,5,Excellent,What a beautiful sunscreen if sweating too muc...,What a beautiful sunscreen if sweating too muc...
4,5,Fabulous!,Good 😊👍🏻READ MORE,Good 😊👍🏻


In [ ]:
print(df['text'][2])

Very nice product go fot it ❤️


In [ ]:
df["text"] = df["text"].astype(str)

In [ ]:
# define selected model pipeline for sentiment analysis task
model_path = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
a = sentiment_task("good😊👍")
a

[{'label': 'positive', 'score': 0.9615573287010193}]

In [ ]:
# Iterrate through our data to run sentiment analysis on
res = {}
for index, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text = row['text']
        myid = index
        res[myid] = dict(sentiment_task(text)[0])
    except RuntimeError:
        print(f'Broke for id {myid}')

100%|██████████| 238/238 [00:19<00:00, 12.11it/s]


In [ ]:
# store results in dataframe
results_df = pd.DataFrame(res).T
results_df.reset_index()
results_df = pd.merge(results_df, df, left_index=True, right_index=True)

In [ ]:
results_df.head()

,label,score,rating,review,comment,text
0,positive,0.980728,5,Perfect product!,Good product 👍👍👍👍👍 thanks FlipkartREAD MORE,Good product 👍👍👍👍👍 thanks Flipkart
1,neutral,0.466284,5,Fabulous!,Nice product...But if I use it in three finger...,Nice product...But if I use it in three finger...
2,positive,0.983053,5,Awesome,Very nice product go fot it ❤️READ MORE,Very nice product go fot it ❤️
3,positive,0.970506,5,Excellent,What a beautiful sunscreen if sweating too muc...,What a beautiful sunscreen if sweating too muc...
4,positive,0.971611,5,Fabulous!,Good 😊👍🏻READ MORE,Good 😊👍🏻


In [ ]:
results_df["label"] = results_df["label"].astype(str)
results_df.head()

,label,score,rating,review,comment,text
0,positive,0.980728,5,Perfect product!,Good product 👍👍👍👍👍 thanks FlipkartREAD MORE,Good product 👍👍👍👍👍 thanks Flipkart
1,neutral,0.466284,5,Fabulous!,Nice product...But if I use it in three finger...,Nice product...But if I use it in three finger...
2,positive,0.983053,5,Awesome,Very nice product go fot it ❤️READ MORE,Very nice product go fot it ❤️
3,positive,0.970506,5,Excellent,What a beautiful sunscreen if sweating too muc...,What a beautiful sunscreen if sweating too muc...
4,positive,0.971611,5,Fabulous!,Good 😊👍🏻READ MORE,Good 😊👍🏻


In [ ]:
# label different results as reqd
pos = results_df[results_df["label"] == "positive"]
neg = results_df[results_df["label"] == "negative"]
neu = results_df[results_df["label"] == "neutral"]
neg.head()

,label,score,rating,review,comment,text
7,negative,0.883473,2,Could be way better,"It's too oily for oily skin, has a greasy effe...","It's too oily for oily skin, has a greasy effe..."
8,negative,0.919028,2,Waste of money!,I had such high expectations frm this product ...,I had such high expectations frm this product ...
11,negative,0.935986,5,Hated it!,Leaves white cast on the face so bad. Just was...,Leaves white cast on the face so bad. Just was...
13,negative,0.840387,5,Classy product,"It's too gd for skin,READ MORE","It's too gd for skin,"
14,negative,0.911385,5,Horrible,Would not recommend for oily skin or even norm...,Would not recommend for oily skin or even norm...


In [ ]:
path = f'/content/drive/MyDrive/Colab Notebooks/output/fkart_ss.csv'
results_df.to_csv(path, index=False)

In [ ]:
pos.head()
path = f'/content/drive/MyDrive/Colab Notebooks/output/fkss_pos.csv'
pos.to_csv(path, index=False)

In [ ]:
neg.head()
path = f'/content/drive/MyDrive/Colab Notebooks/output/fkss_neg.csv'
neg.to_csv(path, index=False)